In [ ]:
%pip install pwhl_api --upgrade
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from pwhl_api.endpoints.gamesummary import GameSummary
from pwhl_api.endpoints.playercareerstats import PlayerCareerStats
from pwhl_api.static.players import get_player

# Configure pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Access Game Summary Data
Get comprehensive information about a specific game including team stats, player performance, and penalties.

In [2]:
# Get summary for a specific game
game_summary = GameSummary(game_id="227")
dfs = game_summary.get_data_frames()

dfs.keys()

dict_keys(['details', 'penalties', 'homeTeam', 'visitingTeam', 'periods', 'mostValuablePlayers'])

In [3]:
# View team information and game score
home_teams_dict, away_teams_dict = dfs['homeTeam'], dfs['visitingTeam']
print(home_teams_dict.keys())
print(home_teams_dict['TeamInfo'])
print(away_teams_dict['TeamInfo'])
print(dfs['details'])

dict_keys(['TeamInfo', 'TeamStats', 'SkaterStats', 'GoalieStats', 'TeamRecord'])
  id              name     city  nickname abbreviation  \
0  6  Toronto Sceptres  Toronto  Sceptres          TOR   

                                             logo divisionName  
0  https://assets.leaguestat.com/pwhl/logos/6.png         PWHL  
  id               name      city  nickname abbreviation  \
0  3  Montréal Victoire  Montréal  Victoire          MTL   

                                             logo divisionName  
0  https://assets.leaguestat.com/pwhl/logos/3.png         PWHL  
    id                          date gameNumber                        venue  \
0  227  Wednesday, December 17, 2025         18  Scotiabank Centre | Halifax   

  attendance    startTime       endTime duration  \
0      10438  7:46 pm EST  10:27 pm EST     2:41   

                                       gameReportUrl  \
0  https://lscluster.hockeytech.com/game_reports/...   

                                     textB

In [4]:
dfs['periods']['goals']

,1,2,3,OT1,SO,T
TOR,1,0,0,0,0,1
MTL,1,0,0,0,1,2


In [5]:
# Analyze period-by-period shooting
periods_dict = dfs['periods']

# Convert columns to numeric
periods_dict['shots']

,1,2,3,OT1,T
TOR,12,12,10,2,36
MTL,8,10,11,2,31


In [6]:
# Analyze skater performance
home_skaters_df = dfs['homeTeam']['SkaterStats']
home_skaters_df['team'] = dfs['homeTeam']['TeamInfo']['name'].values[0]
away_skaters_df = dfs['visitingTeam']['SkaterStats']
away_skaters_df['team'] = dfs['visitingTeam']['TeamInfo']['name'].values[0]
skaters_df = pd.concat([home_skaters_df, away_skaters_df], ignore_index=True)
skaters_df

,player_id,name,position,goals,assists,points,penaltyMinutes,plusMinus,faceoffAttempts,faceoffWins,shots,hits,blockedShots,toi,team
0,68,Kali Flanagan,RD,0,0,0,0,0,0,0,6,0,5,24:46,Toronto Sceptres
1,116,Clair DeGeorge,C,0,0,0,2,0,0,0,0,0,0,1:23,Toronto Sceptres
2,63,Daryl Watts,LW,0,0,0,0,-1,0,0,6,0,1,16:47,Toronto Sceptres
3,317,Kiara Zanon,LW,0,0,0,0,0,0,0,0,0,2,14:34,Toronto Sceptres
4,74,Allie Munroe,LD,0,0,0,0,1,0,0,1,2,3,22:59,Toronto Sceptres
5,67,Renata Fast,RD,0,0,0,2,-1,0,0,1,1,2,24:53,Toronto Sceptres
6,56,Savannah Harmon,LD,0,0,0,0,-1,0,0,3,0,2,21:06,Toronto Sceptres
7,44,Ella Shelton,LD,0,0,0,0,0,0,0,3,2,2,22:07,Toronto Sceptres
8,65,Jesse Compher,RW,0,0,0,0,0,2,0,3,0,2,18:55,Toronto Sceptres
9,313,Sara Hjalmarsson,LW,0,0,0,0,1,0,0,2,1,0,10:34,Toronto Sceptres


In [7]:
# Find top performers in the game
top_performers = skaters_df.nlargest(10, 'points')[['name', 'team', 'goals', 'assists', 'points', 'shots']]
print("\nTop 10 Point Getters:")
print(top_performers.to_string())


Top 10 Point Getters:
                  name               team  goals  assists  points  shots
13    Clara Van Wieren   Toronto Sceptres      0        1       1      0
17          Emma Woods   Toronto Sceptres      0        1       1      0
18       Anna Kjellbin   Toronto Sceptres      1        0       1      2
19          Maya Labad  Montréal Victoire      1        0       1      1
21          Kati Tabin  Montréal Victoire      0        1       1      3
26  Kaitlin Willoughby  Montréal Victoire      0        1       1      1
0        Kali Flanagan   Toronto Sceptres      0        0       0      6
1       Clair DeGeorge   Toronto Sceptres      0        0       0      0
2          Daryl Watts   Toronto Sceptres      0        0       0      6
3          Kiara Zanon   Toronto Sceptres      0        0       0      0


In [8]:
# Analyze team-specific performance
team_names = skaters_df['team'].unique()

for team in team_names:
    team_skaters = skaters_df[skaters_df['team'] == team]
    print(f"\n{team} Skaters:")
    print(f"  Total goals: {team_skaters['goals'].sum():.0f}")
    print(f"  Total assists: {team_skaters['assists'].sum():.0f}")
    print(f"  Total shots: {team_skaters['shots'].sum():.0f}")
    print(f"  Total players: {len(team_skaters)}")
    
    # Top scorer for this team
    top = team_skaters.nlargest(1, 'points')
    if not top.empty:
        print(f"  Leading scorer: {top.iloc[0]['name']} ({top.iloc[0]['points']:.0f} points)")


Toronto Sceptres Skaters:
  Total goals: 1
  Total assists: 2
  Total shots: 36
  Total players: 19
  Leading scorer: Clara Van Wieren (1 points)

Montréal Victoire Skaters:
  Total goals: 1
  Total assists: 2
  Total shots: 31
  Total players: 19
  Leading scorer: Maya Labad (1 points)


## 2. Analyze Penalties
Review penalties, infractions, and player discipline in the game.

In [9]:
# View penalties data
if 'penalties' in dfs and not dfs['penalties'].empty:
    penalties_df = dfs['penalties']
    print(f"Total penalties in game: {len(penalties_df)}")
    print(f"\nColumns: {list(penalties_df.columns)}")
    
    if len(penalties_df) > 0:
        print("\nSample penalties:")
        print(penalties_df[['period_id', 'time', 'against_team_abbreviation', 'taken_by_name', 'description', 'minutes']].head(10).to_string())
else:
    print("No penalties recorded in this game (clean game!)")

Total penalties in game: 5

Columns: ['game_penalty_id', 'period_id', 'time', 'description', 'against_team_id', 'against_team_abbreviation', 'minutes', 'taken_by_id', 'taken_by_name', 'is_power_play', 'is_bench']

Sample penalties:
  period_id   time against_team_abbreviation   taken_by_name   description  minutes
0         1  19:27                       TOR    Emma Maltais      Slashing        2
1         2  10:27                       TOR  Clair DeGeorge      Boarding        2
2         2  13:52                       TOR    Emma Maltais       Hooking        2
3         2  17:01                       TOR     Emma Gentry  Interference        2
4         4  02:27                       TOR     Renata Fast  Interference        2


In [10]:
# Count penalties by team and type
if 'penalties' in dfs and not dfs['penalties'].empty:
    penalties_df = dfs['penalties']
    
    # Group by team
    penalties_by_team = penalties_df.groupby('against_team_abbreviation').size()
    print("Penalties by Team:")
    print(penalties_by_team)
    
    # Analyze penalty types if available
    if 'description' in penalties_df.columns:
        print("\nPenalty Types:")
        print(penalties_df['description'].value_counts())

Penalties by Team:
against_team_abbreviation
TOR    5
dtype: int64

Penalty Types:
description
Interference    2
Slashing        1
Boarding        1
Hooking         1
Name: count, dtype: int64


## 3. Goaltender Analysis
Analyze goaltender performance including saves and goals against.

In [11]:
# Get goaltender statistics
home_goalies_df = dfs['homeTeam']['GoalieStats']
home_goalies_df['team'] = dfs['homeTeam']['TeamInfo']['name'].values[0]
away_goalies_df = dfs['visitingTeam']['GoalieStats']
away_goalies_df['team'] = dfs['visitingTeam']['TeamInfo']['name'].values[0]
goalies_df = pd.concat([home_goalies_df, away_goalies_df], ignore_index=True)
print(f"Total goalies: {len(goalies_df)}")
print(f"\nColumns: {list(goalies_df.columns)}")

# Convert numeric columns
for col in ['saves', 'goals_against', 'shots_against']:
    if col in goalies_df.columns:
        goalies_df[col] = pd.to_numeric(goalies_df[col], errors='coerce')

print("\nGoaltender Performance:")
print(goalies_df[['name', 'team', 'saves', 'goalsAgainst', 'shotsAgainst']].to_string())

Total goalies: 4

Columns: ['player_id', 'name', 'position', 'goals', 'assists', 'points', 'penaltyMinutes', 'plusMinus', 'faceoffAttempts', 'faceoffWins', 'timeOnIce', 'shotsAgainst', 'goalsAgainst', 'saves', 'team']

Goaltender Performance:
                 name               team  saves  goalsAgainst  shotsAgainst
0         Raygan Kirk   Toronto Sceptres     30             1            31
1        Elaine Chuli   Toronto Sceptres      0             0             0
2   Sandra Abstreiter  Montréal Victoire      0             0             0
3  Ann-Renée Desbiens  Montréal Victoire     35             1            36


In [12]:
# Calculate save percentage
if 'shotsAgainst' in goalies_df.columns and 'saves' in goalies_df.columns:
    goalies_df['save_pct'] = goalies_df['saves'] / goalies_df['shotsAgainst']
    
    print("\nGoaltender Efficiency:")
    print(goalies_df[['name', 'team', 'save_pct', 'goalsAgainst']].loc[~goalies_df['save_pct'].isna()].to_string())


Goaltender Efficiency:
                 name               team  save_pct  goalsAgainst
0         Raygan Kirk   Toronto Sceptres  0.967742             1
3  Ann-Renée Desbiens  Montréal Victoire  0.972222             1
